In [1]:
from tbmetapoppy import *
import epyc
import ConfigParser
import cProfile

In [2]:
experiment_name = 'profile'
reps = 5

In [3]:
# Create a notebook for results and a lab in which to run the experiments
nb = epyc.JSONLabNotebook('experiments/' + experiment_name + '/output.json', create=True, 
                          description=experiment_name + ' experiment')
lab = epyc.Lab(nb)

In [4]:
# Import config file
config = ConfigParser.ConfigParser()
config.read('experiments/' + experiment_name + '/config.ini')

['experiments/profile/config.ini']

In [5]:
network_config = {}
network_config[PulmonaryNetwork.TOPOLOGY] = config.get('Network_Build', 'topology')
network_config[PulmonaryNetwork.BOUNDARY] = [[float(a) for a in n[1:-1].split(",")]
                                              for n in config.get('Network_Build', 'boundary').split(":")]
network_config[PulmonaryNetwork.LENGTH_DIVISOR] = config.getfloat('Network_Build', 'length_divisor')
network_config[PulmonaryNetwork.MINIMUM_AREA] = config.getfloat('Network_Build', 'minimum_area')

In [6]:
dynamics = TBDynamics(network_config)
dynamics.set_maximum_time(50)

In [7]:
for section in ['Event_Parameters', 'Network_Parameters']:
    for k,v in config.items(section):
        param = v.split(",")
        if len(param) == 1:
            lab[k] = float(v)
        else:
            minimum, maximum, intervals = [float(n) for n in param]
            lab[k] = numpy.linspace(minimum, maximum, intervals)

In [8]:
cp = cProfile.Profile()

In [9]:
# Run repetitions of the experiment at each point in the parameter space
cp.enable()
lab.runExperiment(epyc.RepeatedExperiment(dynamics, reps))
cp.disable()

In [10]:
cp.print_stats('cumtime')

         117113264 function calls (117026920 primitive calls) in 158.795 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000  158.795   79.397 interactiveshell.py:3036(run_code)
        1    0.000    0.000  158.795  158.795 <ipython-input-9-81461757ca51>:3(<module>)
        1    0.000    0.000  158.795  158.795 lab.py:135(runExperiment)
      6/1    0.000    0.000  158.737  158.737 experiment.py:134(run)
        1    0.000    0.000  158.737  158.737 repeatedexperiment.py:38(do)
        5   15.902    3.180  158.666   31.733 dynamics.py:152(do)
   998914    4.023    0.000   88.789    0.000 network.py:107(update_patch)
   998785    0.826    0.000   84.067    0.000 dynamics.py:127(<lambda>)
   998785   28.802    0.000   83.241    0.000 dynamics.py:39(_propagate_updates)
 24977740   15.361    0.000   45.735    0.000 event.py:19(calculate_rate_at_patch)
   499261    0.870    0.000   45.390    0.000 cell_

In [11]:
cp.print_stats('tottime')

         117113264 function calls (117026920 primitive calls) in 158.795 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   998785   34.232    0.000   34.232    0.000 {method 'multinomial' of 'mtrand.RandomState' objects}
   998785   28.802    0.000   83.241    0.000 dynamics.py:39(_propagate_updates)
        5   15.902    3.180  158.666   31.733 dynamics.py:152(do)
 24977740   15.361    0.000   45.735    0.000 event.py:19(calculate_rate_at_patch)
 30972400    8.913    0.000    8.913    0.000 network.py:89(get_compartment_value)
  9991100    7.653    0.000   18.274    0.000 event.py:34(calculate_rate_at_patch)
   998785    7.605    0.000    7.605    0.000 {method 'reduce' of 'numpy.ufunc' objects}
  2997330    4.788    0.000    6.908    0.000 cell_activation.py:13(_calculate_state_variable_at_patch)
   998914    4.023    0.000   88.789    0.000 network.py:107(update_patch)
  6993770    3.216    0.000    5.258    0.000 cell_